## MeLi Data Challenge 2019

This notebook is part of a curated version of my original solution for the MeLi Data Challenge hosted by [Mercado Libre](https://www.mercadolibre.com/) in 2019

The goal of this first challenge was to create a model that would classify items into categories based solely on the item’s title. 

This title is a free text input from the seller that would become the header of the listings.

<div class="alert alert-block alert-info">
<b>Note</b> <p>Only 10% of the data is used in the notebooks to improve the experience.</p>
    <p>Also, only spanish data is used in this notebooks for simplicity reasons only</p>
    <p>In the scripted version, 100% of the data is used to improve results</p>
</div>

### 2 - PreProcess

In this notebook I'm collecting all the pre-processing steps and alternatives applied to the data

In [1]:
import pandas as pd
import unidecode
import re
from gensim.models.phrases import Phraser, Phrases
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm.auto import tqdm
tqdm.pandas()
import nltk
nltk.download('stopwords')
from statics import *
from gensim.models.word2vec import Word2Vec
import numpy as np
import json
from sklearn.model_selection import train_test_split
import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Basla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load Data

It can be downloaded from here:
- [train](https://meli-data-challenge.s3.amazonaws.com/train.csv.gz)

- [test](https://meli-data-challenge.s3.amazonaws.com/test.csv)

In [2]:
# train_data = pd.read_csv('data/train.csv.gz', compression='gzip')
train_data = pd.read_csv('data/sample_train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_data.head()

,Unnamed: 0,title,label_quality,language,category
0,0,Samsung Dex Station,unreliable,spanish,CELLPHONE_USB_AND_AUXILIARY_ADAPTERS
1,1,Reemplazo Actualizado Para La Válvula De Gas D...,unreliable,spanish,IRRIGATION_VALVES
2,2,Vela Led Luz Calida - Promo X 24 U,unreliable,spanish,CANDLES
3,3,Cebra De Felpa Decoración Safari Candy Bar Vin...,unreliable,spanish,STATUES
4,4,Mormaii Monterrey (smormonte) (negro),unreliable,spanish,HAND_FANS


### Clean titles

Many of the following steps should be run separately for each language.

Here, we run all together, but in the final script data will be splitted.

**Step 1**: run each title through unidecode to remove special characters present in spanish and portuguese

In [4]:
train_data.head(50)

,Unnamed: 0,title,label_quality,language,category
0,0,Samsung Dex Station,unreliable,spanish,CELLPHONE_USB_AND_AUXILIARY_ADAPTERS
1,1,Reemplazo Actualizado Para La Válvula De Gas D...,unreliable,spanish,IRRIGATION_VALVES
2,2,Vela Led Luz Calida - Promo X 24 U,unreliable,spanish,CANDLES
3,3,Cebra De Felpa Decoración Safari Candy Bar Vin...,unreliable,spanish,STATUES
4,4,Mormaii Monterrey (smormonte) (negro),unreliable,spanish,HAND_FANS
5,5,Casio Ae 1000 W,unreliable,spanish,WRISTWATCHES
6,6,Patineta Electrica Nueva Hoverboard Rosados Y ...,unreliable,spanish,HOVERBOARDS
7,7,Barril Cerveza Schofferhofer 5l,unreliable,spanish,BEER_KEGS
8,8,Huggies Natural Care Hiper Pack Ellas M X 68,unreliable,spanish,BABY_DIAPERS
9,9,Burro De Arranque Yamaha Ybr,unreliable,spanish,MOTORCYCLE_STARTER_MOTORS


In [5]:
train_data.tail(50)

,Unnamed: 0,title,label_quality,language,category
199950,199950,Botellas Eco Twis 500 Ml #tupperware,unreliable,spanish,SPORT_AND_BAZAAR_BOTTLES
199951,199951,Extractor De Frutas Enteras 1 Litro 800 Watts ...,unreliable,spanish,COTTON_CANDY_MACHINES
199952,199952,"Servilleteros Vinilo, Estilo Retro, De Carton",unreliable,spanish,NAPKIN_HOLDERS
199953,199953,Zapatillas Outdoor Fila Slant Force Hombre On ...,unreliable,spanish,SNEAKERS
199954,199954,Prensa De Hierro Casi Sin Uso. Como Nuevas De ...,unreliable,spanish,BAR_CLAMPS
199955,199955,Mochilitas Y Carteras Golosineras Para Tus Fie...,unreliable,spanish,COMMERCIAL_AND_EVENT_BAGS
199956,199956,Butaca Love 2017,unreliable,spanish,BABY_CAR_SEATS
199957,199957,Memoria Flash Nan Para Tv Led Samsung Serie D5...,unreliable,spanish,MEMORY_CARDS
199958,199958,Rainbow Six Vegas,unreliable,spanish,VIDEO_GAMES
199959,199959,Frapperas Y Hieleras,unreliable,spanish,ICE_BUCKETS


In [6]:
for text in train_data.iloc[[10, 199955]].title.values:
    print(f"\n Original text: {text}")
    print(f"Processed text: {unidecode.unidecode(text)}")


 Original text: Reloj Antiguo De Pared Con Pendulo.
Processed text: Reloj Antiguo De Pared Con Pendulo.

 Original text: Mochilitas Y Carteras Golosineras Para Tus Fiestas ....  
Processed text: Mochilitas Y Carteras Golosineras Para Tus Fiestas ....  


**Step 2**: Clean special cases using RegEx

In [7]:
for string in train_data.head().title.values:
    stringA = re.sub(r"\s{2,}", " ", string)
    stringB = re.sub(r'(.)\1{3,}', r'\1\1', stringA)
    if string != stringB:
        print(f"\n Original text: {string}")
        print(f"Processed text: {stringB}")

In [8]:
# Remove extra spaces and characters repeated more than 3 times in a row
for string in train_data.iloc[[233, 413]].title.values:
    stringA = re.sub(r"\s{2,}", " ", string)
    stringB = re.sub(r'(.)\1{3,}', r'\1\1', stringA)
    if string != stringB:
        print(f"\n Original text: {string}")
        print(f"Processed text: {stringB}")

In [9]:
# Find spatial measures and replace them with a special tag
for string in train_data.head(50).title.values:
    stringB = re.sub(r"((\d)+(\,|\.){0,1}(\d)*( ){0,1}((mts*)|(pulgadas*)|('')|(polegadas*)|(m)|(mms*)|(cms*)|(metros*)|(mtrs*)|(centimetros*))+)(?!(?!x)[A-Za-z])", " <smeasure> ", string, flags=re.I)
    if string != stringB:
        print(f"\n Original text: {string}")
        print(f"Processed text: {stringB}")



 Original text: Pisos Flotantes Kronotex Dynamic 8mm Alemanes Xm2
Processed text: Pisos Flotantes Kronotex Dynamic  <smeasure>  Alemanes Xm2

 Original text: Piso Flotante Euro Tec Premium 8,3 Mm Ac5 Roble Oceano
Processed text: Piso Flotante Euro Tec Premium  <smeasure>  Ac5 Roble Oceano


**Step 3**
Putting it all together (and adding a "few" more expressions):

In [10]:
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    
    string = unidecode.unidecode(string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r'(.)\1{3,}', r'\1\1', string)
    
    string = re.sub(r'\d{7,}', r' <longnumber> ', string)
    string = re.sub(r'\d{7,}', r' <longnumber> ', string)
    string = re.sub(r'(?<=[A-Za-z])[\d,]{10,}', r' <model> ', string)
    string = re.sub(r'[\d,]{10,}', r' <weird> ', string)
    
    # Spatial
    string = re.sub(r"((\d)+(\,|\.){0,1}(\d)*( ){0,1}((mts*)|(pulgadas*)|('')|"
                    "(polegadas*)|(m)|(mms*)|(cms*)|(metros*)|(mtrs*)|(centimetros*))+)"
                    "(?!(?!x)[A-Za-z])", " <smeasure> ", string, flags=re.I)
    string = re.sub(r"(mts)+( ){0,1}((\d)+(\,|\.){0,1}(\d)*)(?![A-Za-z])", " <smeasure> ", string, flags=re.I)
    string = re.sub(r"<smeasure> +[\/x] +<smeasure> +[\/x] +<smeasure>", " <smeasure> ", string, flags=re.I)
    string = re.sub(r"(<smeasure>|(\d)+(\,|\.)*(\d*)) +[\/x] "
                    "+(<smeasure>|(\d)+(\,|\.)*(\d*)) +[\/x] +<smeasure>", " <smeasure> ", string, 
                    flags=re.I)
    string = re.sub(r"<smeasure> +[\/x] +<smeasure>", " <smeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])(<smeasure>|(\d)+(\,|\.)*(\d*)) *[\/x-] *<smeasure>", 
                    " <smeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*))x((\d)+(\,|\.)*(\d*))x +<smeasure>", " <smeasure> ", 
                    string, flags=re.I)
    
    # Electrical
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *amperes", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"(?<!(?<![\dx])[\dA-Za-z])((\d)+(\,|\.)*(\d*)) *amps*", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *mah", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *vol.", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *kw\b", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *v+\b", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *volts*", " <emeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *w(ts)*(?![\dA-Za-z])", " <emeasure> ", 
                    string, flags=re.I)
    
    # Pressure
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *psi", " <pmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *bar", " <pmeasure> ", string, flags=re.I)
    
    # Weights
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *kgs*", " <wmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *kilos*", " <wmeasure> ", string, flags=re.I)
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *g\b", " <wmeasure> ", string, flags=re.I)
    
    # IT
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *tb", " <itmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *gb", " <itmeasure> ", string, flags=re.I)
    
    # Volume
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *cc(?![0-9])", " <vmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *litros*", " <vmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*))litrs*", " <vmeasure> ", string, flags=re.I)
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *l+\b", " <vmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *mls*", " <vmeasure> ", string, flags=re.I)
    string = re.sub(r"(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*)) *ltrs*", " <vmeasure> ", string, flags=re.I)
    
    # Horse power
    string = re.sub(r"\b(?<![\dA-Za-z])(\d)+ *cv\b", " <hpmeasure> ", string, flags=re.I)
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\,|\.)*(\d*))+ *hp\b", " <hpmeasure> ", string, flags=re.I)
    
    # Time
    string = re.sub(r"\b(?<![\dA-Za-z])((\d)+(\:)*(\d*)) *hs*(?![\d])\b", " <time> ", string, flags=re.I)
    
    # Quantity
    string = re.sub(r"\bX\d{1,4}\b", " <quantity> ", string, flags=re.I)
    
    # Money
    string = re.sub(r"(?<![\dA-Za-z])\$ *((\d)+(\,|\.)*(\d*))", " <money> ", string, flags=re.I)
    
    # Dimension (could be smeasure too)
    string = re.sub(r"(((\d)+(\,|\.)*(\d*))+ {0,1}(x|\*){1} {0,1}((\d)+(\,|\.)*(\d*))+)"
                    "+( {0,1}(x|\*){1} {0,1}((\d)+(\,|\.)*(\d*))+)*", " <dimension> ", string, flags=re.I) 
    
    # Resolution
    string = re.sub(r"\b(?<![A-Za-z\-])\d+p\b", " <res> ", string)
    
    # Date
    string = re.sub(r"\b\d{2}-\d{2}-(19\d{2}|20\d{2})\b", " <date> ", string)

    # Model
    string = re.sub(r"(?<!\d{4})[A-Za-z\-]+\d+[A-Za-z\-\.0-9]*", " <model> ", string, flags=re.I)
    string = re.sub(r"[A-Za-z\-\.0-9]*\d+[A-Za-z\-](?!\d{4})", " <model> ", string, flags=re.I)
    string = re.sub(r"<model> \d+", " <model> ", string, flags=re.I)
    string = re.sub(r"\d+ <model>", " <model> ", string, flags=re.I)
    
    # Years
    string = re.sub(r"(?<![A-Za-z0-9])19\d{2}|20\d{2}(?![A-Za-z0-9])", " <year> ", string)
    
    # Numbers
    string = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", string)
    
    # String cleanup
    string = re.sub(r",", " , ", string)
    string = re.sub(r"/", " / ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\*", " * ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"#\S+", " <hashtag> " , string)
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\+", " ", string)
    string = re.sub(r"\d+", " <number> ", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`<>]", " ", string, flags=re.I)
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

In [11]:
for string in train_data.sample(5).title.values:
    print(f"\n Input string: {string}")
    print(f"Clean string: {clean_str(string)}")    


 Input string: Chupetes Ultra Soft 0-6m Philips Avent - Pack 2 Piezas Nena
Clean string: chupetes ultra soft <smeasure> philips avent pack <number> piezas nena

 Input string: Precintos Plasticos Numerados
Clean string: precintos plasticos numerados

 Input string: Medidor De Distancia Laser Glm 20 Bosch 20mts Fernoluz
Clean string: medidor de distancia laser glm <number> bosch <smeasure> fernoluz

 Input string: Faber-castell 7358-113 Lapiz Albrecht Durer Por Unidad Naran
Clean string: faber castell <model> lapiz albrecht durer por unidad naran

 Input string: M/o Cambio De Rulemanes Delanteros P/lado Volkswagen Passat
Clean string: m o cambio de rulemanes delanteros p lado volkswagen passat


**Apply cleaning to entire dataset**

Considering that it is a 20M rows dataset, this process is quite time consuming. That's the reasong why it is performed separately and the results stored.

In [12]:
train_data['clean_title'] = train_data.progress_apply(lambda x : clean_str(x['title']), axis=1)

  0%|          | 0/200000 [00:00<?, ?it/s]

**Save cleaned dataset**

In [13]:
train_data.to_csv('data/sample_clean_train.csv')

### Bigrams

In [14]:
sentences = train_data['clean_title'].astype(str)

In [15]:
# Now that the text is clean, we can split it into words just by splitting on spaces
x_text = [s.split(" ") for s in sentences]

In [16]:
bigram = Phraser(Phrases(x_text, min_count=100, threshold=20))

In [17]:
x_text = [bigram[text] for text in x_text]

### Remove stopwords

In [18]:
def remove_stopwords(sentences, lang='english'):
    try:
        stpwrds = stopwords.words(lang)
    except Exception:
        stpwrds = stopwords.words('spanish')
        
    out_sentences = [[w for w in sentence if w not in stpwrds] for sentence in sentences]

    return out_sentences

In [19]:
x_text = remove_stopwords(x_text, lang='spanish')

### Stemming

In [20]:
def text_stemming(sentences, lang='english'):
    try:
        stemmer = SnowballStemmer(lang)
    except Exception:
        stemmer = SnowballStemmer('spanish')
    
    out_text = [[stemmer.stem(i) for i in text] for text in sentences]

    return out_text

In [21]:
x_text = text_stemming(x_text, lang='spanish')

### Pad sentences
In order to feed the data into a CNN, all input sentences must have the same length

This next step add a padding token or truncate sentences to make all of them of the same lenght

In [22]:
def pad_sentences(sentences, padding_word="<PAD/>", len_sent = None):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    if len_sent is None:
        sequence_length = max(len(x) for x in sentences)
    else:
        sequence_length = len_sent
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        if num_padding >= 0:
            new_sentence = sentence + [padding_word] * num_padding
        else:
            new_sentence = sentence[:sequence_length]
        padded_sentences.append(new_sentence)
    return padded_sentences

In [23]:
x_text = pad_sentences(x_text)

In [24]:
print(x_text[1])

['reemplaz', 'actualiz', 'valvul', 'gas', 'horn', 'bryant', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>']


### Word Embeddings

For this project I used Word2Vec and now it is a good point to calculate the vectors

In [25]:
EMBEDDING_DIM = 300
model = Word2Vec(sentences = x_text, vector_size=EMBEDDING_DIM, sg=1, window=7, min_count=20, seed=42, workers=8)

We should also save the weights and vocabulary to be used later in the CNN

In [26]:
weights = model.wv.vectors
np.save(open('data/embeddings.npz', 'wb'), weights)
vocab = model.wv.key_to_index 
with open('data/map.json', 'w') as f:
    f.write(json.dumps(vocab))

### Build input data

Now that we have the veight vectors and the word indices, we can replace each word with an index to be used as an input to the CNN

In [27]:
train_data['sentences_padded'] = x_text

In [28]:
def load_vocab(vocab_path='/map.json', path=''):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(path+vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [29]:
def build_input_data(df, path=''):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    word2idx, idx2word = load_vocab(vocab_path='/map.json', path=path)
    df['input_data'] = df.apply(lambda x : np.array([word2idx[word] if word in word2idx.keys() else word2idx["<PAD/>"] for word in x['sentences_padded']], dtype=np.int32), axis=1)
    return df

In [30]:
train_data = build_input_data(train_data, path='data')

### Split data into train and test
Since we are using a 1% of the data only, some categories won't have enough data.

We will drop them for now abut this won't be included in the final code

In [31]:
# Clean title column is no longer needed
train_data.drop(columns=['clean_title'], inplace=True)

In [32]:
train_data = train_data[~train_data.category.isin(train_data.category.value_counts()[train_data.category.value_counts() < 2].index)]

In [33]:
train_df, test_df = train_test_split(train_data,test_size=0.1, stratify=train_data['category'])

### Save the data

Save the data to be used in the models

In [34]:
for each in train_df['sentences_padded']:
    len_sent = len(each)
    break

In [35]:
joblib.dump(len_sent, './data/len_sent.h5')      
train_df.to_pickle('./data/df.pkl')
test_df.to_pickle('./data/df_test.pkl')